# Voorbeeldopdracht week 10
Deze week gaan we de analyses van week 1 voortzetten door actief op zoek te gaan naar verbanden en analyses te doen die moeten leiden tot het beantwoorden van een onderzoeksvraag.
We beginnen met het importeren van een dataset

In [2]:
import pandas as pd
data = pd.read_excel('Dataset1 week 10.xlsx')
data.head(10)

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Date_becoming_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,Responded to deal
0,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,1
1,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,0
2,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,0
3,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,0
4,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,0
5,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,42,98,0,42,14,0
6,1971,Graduation,Divorced,55635.0,0,1,2012-11-13,34,235,65,164,50,49,27,0
7,1985,PhD,Married,33454.0,1,0,2013-05-08,32,76,10,56,3,1,23,0
8,1974,PhD,Together,30351.0,1,0,2013-06-06,19,14,0,24,3,3,2,1
9,1950,PhD,Together,5648.0,1,1,2014-03-13,68,28,0,6,1,1,13,0


Dit is een dataset met klantgegevens. De kolommen zijn
* Geboortejaar
* Opleidingsniveau
* Burgelijke staat
* Inkomen
* Aantal kinderen in huis
* Aantal tieners in huis
* Datum waarop een klant voor het eerst klant is geworden
* Aantal dagen sinds laatste aankoop
* Hoeveelheid geld uitgegeven aan wijn in de laatste 2 jaar
* Hoeveelheid geld uitgegeven aan fruit de afgelopen 2 jaar
* Hoeveelheid geld uitgegeven aan vlees de afgelopen 2 jaar
* Hoeveelheid geld uitgegeven aan vis de afgelopen 2 jaar
* Hoeveelheid geld uitgegeven aan snoep de afgelopen 2 jaar
* Hoeveelheid geld uitgegeven aan goud de afgelopen 2 jaar
* Indicator of de persoon de laatste keer mee deed aan een actie.

Het bedrijf wil uiteindelijk weten welke klanten potentieel gevoelig zijn voor een actie


Bij een dergelijke analyse kunnen allerlei deelvragen bepaald worden. Hieronder wordt een aantal voorbeelden uitgewerkt

1. Geven mensen met een hoger opleidingsniveau meer uit?

In [ ]:
# Groeperen per opleidingsniveau. Som van uitgaven
data.groupby(['Education'])['MntWines', 'MntFruits', 'MntMeatProducts', 
                                           'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'].mean()


C:\Users\thoma\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds
Education,,,,,,
2n Cycle,198.182266,28.955665,141.256158,47.482759,34.251232,46.399015
Basic,7.240741,11.111111,11.444444,17.055556,12.111111,22.833333
Graduation,284.268855,30.774623,179.488909,43.149956,31.367347,50.849157
Master,333.075676,21.654054,163.378378,32.100000,21.175676,40.397297
PhD,404.495885,20.049383,168.602881,26.728395,20.222222,32.310700


Zoals je ziet best wel grote verschillen

2. Geven huishoudens met kinderen meer uit aan snoep?

In [ ]:
data.groupby(['Kidhome', 'Teenhome'])['MntSweetProducts'].mean().reset_index()


,Kidhome,Teenhome,MntSweetProducts
0,0,0,53.061129
1,0,1,28.700800
2,0,2,19.133333
3,1,0,9.850895
4,1,1,7.521390
5,1,2,11.909091
6,2,0,8.058824
7,2,1,2.870968


Opvallend genoeg zien we dat dat niet het geval is, kunnen we verklaren hoe dat komt? Daarvoor kunnen we kijken hoe de groep zonder kinderen op andere indicatoren verschilt van de andere groepen. Hiervoor hebben we ene nieuwe kolom nodig: wel of geen kinderen

In [ ]:
# Nieuwe kolom maken
data['Kinderen'] = data.apply(lambda x: 1 if x['Kidhome'] + x['Teenhome'] > 0 else 0, axis=1)
# Gemiddelde van alle kolommen bekijken
data.groupby('Kinderen').mean()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,Responded to deal
Kinderen,,,,,,,,,,,,
0,1968.583072,65705.565561,0.000000,0.000000,49.086207,487.305643,52.357367,372.609718,76.547022,53.061129,64.148903,0.264890
1,1968.894507,46865.626027,0.621099,0.707865,49.118602,230.908240,15.925718,85.045568,21.985019,16.709114,36.006242,0.102996


Een aantal dingen valt op. We zien dat mensen zonder kinderen gemiddeld een hoger inkomen hebben en dat ze in algemene zin dus veel meer uitgaven. Dit valt ook op als we kijken naar het opleidingsniveau

In [ ]:
pd.pivot_table(data, index='Education', columns=['Kinderen'], values='Year_Birth', aggfunc='count')

Kinderen,0,1
Education,,
2n Cycle,64,139
Basic,17,37
Graduation,321,806
Master,102,268
PhD,134,352
